In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from typing import List, Tuple
import matplotlib.pyplot as plt
import numpy as np
import fdb
import loompy
import shoji
from tqdm import trange
import logging
import sys
#from rich.progress import track
%load_ext line_profiler
logging.basicConfig(stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s', level=20)

In [2]:
db = shoji.connect()
db

,Contents
eel,"2 workspaces, 0 dimensions, 0 tensors"
images,"0 workspaces, 1 dimensions, 1 tensors"
refdb,"2 workspaces, 0 dimensions, 0 tensors"
scRNA,"0 workspaces, 2 dimensions, 3 tensors"
test,"0 workspaces, 2 dimensions, 42 tensors"


In [12]:
db.refdb.human_development._from_loom("/Users/stelin/Dropbox (Linnarsson Group)/Temp/Allbrain.loom")

100%|██████████| 812/812 [19:01:25<00:00, 84.34s/it]      


In [13]:
ws = db.refdb.human_development
ws

Workspace with 0 workspaces, 2 dimensions and 97 tensors:
  cells <Dimension of variable shape>
  genes <Dimension of shape 33538>
  Accession <Tensor Accession dtype='string' dims=('genes',), shape=(33538,)>
  Age <Tensor Age dtype='string' dims=('cells',), shape=(1622088,)>
  All_fc_analysis_id <Tensor All_fc_analysis_id dtype='string' dims=('cells',), shape=(1622088,)>
  Ambiguous <Tensor Ambiguous dtype='uint16' dims=('cells', 'genes'), shape=(1622088, 33538)>
  Analysis <Tensor Analysis dtype='string' dims=('cells',), shape=(1622088,)>
  Bplength <Tensor Bplength dtype='int64' dims=('cells',), shape=(1622088,)>
  Cdnalibcomment <Tensor Cdnalibcomment dtype='string' dims=('cells',), shape=(1622088,)>
  Cdnalibok <Tensor Cdnalibok dtype='string' dims=('cells',), shape=(1622088,)>
  CellCycle <Tensor CellCycle dtype='float64' dims=('cells',), shape=(1622088,)>
  CellCycle_G1 <Tensor CellCycle_G1 dtype='float64' dims=('cells',), shape=(1622088,)>
  CellCycle_G2M <Tensor CellCycle_G2M dtype='float64' dims=('cells',), shape=(1622088,)>
  CellCycle_S <Tensor CellCycle_S dtype='float64' dims=('cells',), shape=(1622088,)>
  CellID <Tensor CellID dtype='string' dims=('cells',), shape=(1622088,)>
  Cellconc <Tensor Cellconc dtype='string' dims=('cells',), shape=(1622088,)>
  Chemistry <Tensor Chemistry dtype='string' dims=('cells',), shape=(1622088,)>
  Chromosome <Tensor Chromosome dtype='string' dims=('genes',), shape=(33538,)>
  Clusterdensity <Tensor Clusterdensity dtype='int64' dims=('cells',), shape=(1622088,)>
  Clusterpf <Tensor Clusterpf dtype='int64' dims=('cells',), shape=(1622088,)>
  Clusters <Tensor Clusters dtype='int64' dims=('cells',), shape=(1622088,)>
  ClustersModularity <Tensor ClustersModularity dtype='int64' dims=('cells',), shape=(1622088,)>
  ClustersSurprise <Tensor ClustersSurprise dtype='int64' dims=('cells',), shape=(1622088,)>
  Comment <Tensor Comment dtype='string' dims=('cells',), shape=(1622088,)>
  Config <Tensor Config dtype='string' dims=(), shape=()>
  CreationDate <Tensor CreationDate dtype='string' dims=('cells',), shape=(1622088,)>
  CreationDate_gobal <Tensor CreationDate_gobal dtype='string' dims=(), shape=()>
  Datecaptured <Tensor Datecaptured dtype='string' dims=('cells',), shape=(1622088,)>
  Dateseqlib <Tensor Dateseqlib dtype='string' dims=('cells',), shape=(1622088,)>
  Donor <Tensor Donor dtype='string' dims=('cells',), shape=(1622088,)>
  DoubletFinderFlag <Tensor DoubletFinderFlag dtype='int64' dims=('cells',), shape=(1622088,)>
  DoubletFinderScore <Tensor DoubletFinderScore dtype='float64' dims=('cells',), shape=(1622088,)>
  Editat <Tensor Editat dtype='string' dims=('cells',), shape=(1622088,)>
  Editby <Tensor Editby dtype='string' dims=('cells',), shape=(1622088,)>
  End <Tensor End dtype='int64' dims=('genes',), shape=(33538,)>
  Expression <Tensor Expression dtype='float32' dims=('cells', 'genes'), shape=(1622088, 33538)>
  Finalconc <Tensor Finalconc dtype='string' dims=('cells',), shape=(1622088,)>
  Gene <Tensor Gene dtype='string' dims=('genes',), shape=(33538,)>
  Id <Tensor Id dtype='int64' dims=('cells',), shape=(1622088,)>
  Idxset <Tensor Idxset dtype='string' dims=('cells',), shape=(1622088,)>
  Inputmrna <Tensor Inputmrna dtype='float64' dims=('cells',), shape=(1622088,)>
  IsCycling <Tensor IsCycling dtype='uint8' dims=('cells',), shape=(1622088,)>
  LOOM_SPEC_VERSION <Tensor LOOM_SPEC_VERSION dtype='string' dims=(), shape=()>
  Label <Tensor Label dtype='string' dims=('cells',), shape=(1622088,)>
  MT_ratio <Tensor MT_ratio dtype='float64' dims=('cells',), shape=(1622088,)>
  MeanTotalUMI <Tensor MeanTotalUMI dtype='float64' dims=('cells',), shape=(1622088,)>
  Method <Tensor Method dtype='string' dims=('cells',), shape=(1622088,)>
  Molarity <Tensor Molarity dtype='string' dims=('cells',), shape=(1622088,)>
  MultilevelMarkers <Tensor MultilevelMarkers dtype='float64' dims=('genes', 4), shape=(33538, 4)>
  NGenes <Tensor NGenes dtype='float64' dims=('cells',)

In [ ]:
gp = ws.cells.groupby("Clusters")
gp.Tissue("mode")
gp.TotalUMI("median")